<a href="https://colab.research.google.com/github/diegoesquivel09/ejerciciosbsm/blob/main/Eurosourcing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mip

from mip import *


### Definir objeto modelo y los datos


In [18]:
#Crear el modelo

model = Model()

# Datos del problema. --

mercados = ['Madrid', 'París', 'Berlín', 'Roma', 'Amsterdam']
costo_produccion_local = {'Madrid': 460, 'París': 515, 'Berlín': 505, 'Roma': 480, 'Amsterdam': 490}
demanda = {'Madrid': 420, 'París': 1310, 'Berlín': 1250, 'Roma': 930, 'Amsterdam': 870}
#Agregar costo transporte terrestre 0 entre iguales ciudades
costo_transporte_terrestre = {
    ('Madrid', 'París'): 10, ('Madrid', 'Berlín'): 35, ('Madrid', 'Roma'): 25, ('Madrid', 'Amsterdam'): 25, ('Madrid', 'Madrid'): 0,
    ('París', 'Berlín'): 15, ('París', 'Roma'): 20, ('París', 'Amsterdam'): 10, ('París', 'Madrid'): 10, ('París', 'París'): 0,
    ('Berlín', 'Madrid'): 35,('Berlín', 'París'): 15,('Berlín', 'Roma'): 25, ('Berlín', 'Amsterdam'): 10, ('Berlín', 'Berlín'): 0,
    ('Roma', 'Madrid'): 25, ('Roma', 'París'): 20, ('Roma', 'Berlín'): 25, ('Roma', 'Amsterdam'): 23, ('Roma', 'Roma'): 0,
    ('Amsterdam', 'Madrid'): 25, ('Amsterdam', 'París'): 10, ('Amsterdam', 'Berlín'): 10, ('Amsterdam', 'Roma'): 23, ('Amsterdam', 'Amsterdam'): 0,
}
costo_transporte_maritimo_china = {'Madrid': 9, 'París': 8.25, 'Berlín': 9, 'Roma': 10, 'Amsterdam': 6}
costo_produccion_china = 440  # Costo unitario de producción en China


#Definir variables

In [19]:
### Definir variables de decisión

comprar_china = {m: model.add_var(var_type=CONTINUOUS, lb=0) for m in mercados} ## Producos OVIS a cada mercado/almacen
comprar_local = {m: model.add_var(var_type=CONTINUOUS, lb=0) for m in mercados} ## Productos producidos en DOMES en un mercado
envios = {(o, d): model.add_var(var_type=CONTINUOUS, lb=0) for o in mercados for d in mercados} ## Envíos desde un almacen a otro

produce_local = {m: model.add_var(var_type=BINARY) for m in mercados} ##


#Restricciones de demanda

In [20]:
for m in mercados:
    model.add_constr(comprar_china[m] + comprar_local[m] + xsum(envios[(o, m)] for o in mercados) == demanda[m] + xsum(envios[(m, d)] for d in mercados))

model.add_constr(xsum(produce_local[m] for m in mercados) == 1) ## Solo se puede producir en un solo lugar de DOMES

model.add_constr (comprar_local[m] >= 150) #verificar el mínimo que se produce internamente

model.add_constr (comprar_local[m] <= produce_local[m] * 5000 ) #asociar produce local con compra local




# Función Objetivo

In [21]:
model.objective= minimize (
    xsum(comprar_china[m] * costo_produccion_china for m in mercados) +
    xsum(comprar_local[m] * costo_produccion_local[m] for m in mercados) +
    xsum(comprar_china[m] * costo_transporte_maritimo_china[m] for m in mercados) +
    xsum(envios[o, d] * costo_transporte_terrestre[o,d] for o in mercados for d in mercados) ##si toma igual ciudad para origen y destino debería tomar valor 0
)



Resolver el model

In [22]:
status= model.optimize()
print("Estado:", status)

print("Cantidad a comprar desde China:")
for m in mercados:
    print(f"{m}: {comprar_china[m].x}")

print("\nCantidad a comprar localmente:")
for m in mercados:
    print(f"{m}: {comprar_local[m].x}")

print("Costo total de producción y transporte:",  model.objective_value)

Estado: OptimizationStatus.OPTIMAL
Cantidad a comprar desde China:
Madrid: 420.0
París: 1310.0
Berlín: 1250.0
Roma: 930.0
Amsterdam: 720.0

Cantidad a comprar localmente:
Madrid: 0.0
París: 0.0
Berlín: 0.0
Roma: 0.0
Amsterdam: 150.0
Costo total de producción y transporte: 2150157.5


In [13]:
total= sum(demanda.values())
print(total)

4780
